# Library

In [ ]:
%tensorflow_version 1.x
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
%tensorflow_version 1.x
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/xlnet-paper/")

In [ ]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

In [ ]:
!pip3 install sentencepiece

     |████████████████████████████████| 1.1MB 3.3MB/s 


In [ ]:
# INFO:tensorflow:Best result | eval_accuracy 0.32539311051368713 | eval_loss 3.165174722671509 | global_step 80000 | loss 3.1305904388427734 | path gs://zalo_cuongdv2/paper/xlnet-base-keywords/model.ckpt-80000 | step 80000 | 

In [ ]:
Best result | eval_accuracy 0.3568417429924011 | eval_loss 2.895155191421509 | global_step 60000 | loss 2.8858845233917236 | path gs://zalo_cuongdv2/paper/xlnet-base-keywords-cased/model.ckpt-60000 | step 60000 |

# Main code

In [ ]:
FEATURES = ["title", "keywords", "abstract"] # ["title"]
if "abstract" in FEATURES: max_seq_length = 512
elif len(FEATURES) == 2: max_seq_length = 256
else: max_seq_length = 128
fff = ",".join(FEATURES)
DATA_DIR = "../data/cs-paper/"
OUTPUT_DIR = "gs://paper/xlnet-large-cs-{}/".format("+".join(FEATURES))
PRETRAIN_MODEL_DIR = "gs://bert-eng/xlnet-large-cased"

!python3 run_classifier_cs.py \
  --features=$fff \
  --overwrite_data=True \
  --use_tpu=True \
  --tpu=$TPU_ADDRESS \
  --do_train=True \
  --do_eval=True \
  --eval_all_ckpt=True \
  --data_dir=$DATA_DIR \
  --output_dir=$OUTPUT_DIR \
  --model_dir=$OUTPUT_DIR \
  --uncased=True \
  --spiece_model_file=spiece.model \
  --model_config_path=$PRETRAIN_MODEL_DIR/xlnet_config.json \
  --init_checkpoint=$PRETRAIN_MODEL_DIR/xlnet_model.ckpt \
  --max_seq_length=$max_seq_length \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --num_hosts=1 \
  --num_core_per_host=8 \
  --learning_rate=5e-5 \
  --num_train_epochs=10.0 \
  --warmup_proportion=0.1 \
  --save_steps=1000 \
  --iterations=1000

In [ ]:
import json
with tf.io.gfile.GFile(os.path.join(OUTPUT_DIR, "paper.logits.json"), "r") as infile:
    probabilities = json.load(infile)

import numpy as np
probabilities = np.asarray(probabilities)

from run_classifier_cs import PaperProcessor
journals = PaperProcessor().get_labels()
journal_to_idx = {journal:ii for ii, journal in enumerate(journals)}

y_valid = []
for item in open(os.path.join(DATA_DIR, "valid.jsonl")):
    item = json.loads(item)
    if "keywords" in FEATURES:
        if "keywords" not in item or type(item['keywords']) != str or \
            len(item['keywords']) < 1: continue
    y_valid.append(journal_to_idx[item['journal'].lower()])
y_valid = np.asarray(y_valid)

print(np.mean(np.repeat(y_valid, 1).reshape(-1,1) == np.argsort(probabilities, axis=-1)[:,:-2:-1])*1)
print(np.mean(np.repeat(y_valid, 3).reshape(-1,3) == np.argsort(probabilities, axis=-1)[:,:-4:-1])*3)
print(np.mean(np.repeat(y_valid, 5).reshape(-1,5) == np.argsort(probabilities, axis=-1)[:,:-6:-1])*5)
print(np.mean(np.repeat(y_valid, 10).reshape(-1,10) == np.argsort(probabilities, axis=-1)[:,:-11:-1])*10)